# AdaBoostClassifier with RandomForest Hyper Parameter Tuning using GridSearchCV

In [56]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV, RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, BaggingClassifier, AdaBoostClassifier 
from sklearn.metrics import accuracy_score, classification_report, plot_confusion_matrix, precision_score, roc_auc_score
from sklearn.pipeline import Pipeline

import category_encoders as ce

In [57]:
master_df = pd.read_csv('SAMPLE1.csv')

In [58]:
master_df.head()

,Primary Key,Created,Mileage In,Work Order,Model,Line Type,Service Category,Service Package,Description,Definition,...,Quantity,Total,Discount,Net Total,ContactID,ServiceItemID,InvoiceID,labels,Make,Year
0,1,12/28/2018 10:18,33320,49,civic,Material,"Oil, Lube & Preventive Maintenance Service",MINOR Preventive Maintenance Service (MEDIUM S...,OIL FILTER HONDA,Engine Oil Filter,...,1.0,696.43,0.0,696.43,a271834a-1afc-4e62-8c71-638025174c1f,ff01e2a4-d2d5-48ff-8ccc-d5605a653df4,99833bf9-865b-4e2c-9bd1-a0ed0cebea8e,preventive maintenance,honda,2012
1,2,12/28/2018 10:18,33320,49,civic,Material,"Oil, Lube & Preventive Maintenance Service",MINOR Preventive Maintenance Service (MEDIUM S...,ENGINE FLUSHING,NaN,...,1.0,142.86,0.0,142.86,a271834a-1afc-4e62-8c71-638025174c1f,ff01e2a4-d2d5-48ff-8ccc-d5605a653df4,99833bf9-865b-4e2c-9bd1-a0ed0cebea8e,preventive maintenance,honda,2012
2,3,12/28/2018 10:18,33320,49,civic,Material,"Oil, Lube & Preventive Maintenance Service",MINOR Preventive Maintenance Service (MEDIUM S...,BRAKE CLEANER 14oz,NaN,...,1.0,227.68,0.0,227.68,a271834a-1afc-4e62-8c71-638025174c1f,ff01e2a4-d2d5-48ff-8ccc-d5605a653df4,99833bf9-865b-4e2c-9bd1-a0ed0cebea8e,preventive maintenance,honda,2012
3,4,12/28/2018 10:18,33320,49,civic,Labor,"Oil, Lube & Preventive Maintenance Service",MINOR Preventive Maintenance Service (MEDIUM S...,Labor,NaN,...,3.0,1339.29,0.0,1339.29,a271834a-1afc-4e62-8c71-638025174c1f,ff01e2a4-d2d5-48ff-8ccc-d5605a653df4,99833bf9-865b-4e2c-9bd1-a0ed0cebea8e,preventive maintenance,honda,2012
4,5,12/28/2018 10:18,33320,49,civic,Material,"Oil, Lube & Preventive Maintenance Service",MINOR Preventive Maintenance Service (MEDIUM S...,REPSOL ELITE INJECTION 10W40 1L - GAS/DIESEL,Engine Oil,...,4.0,1178.56,0.0,1178.56,a271834a-1afc-4e62-8c71-638025174c1f,ff01e2a4-d2d5-48ff-8ccc-d5605a653df4,99833bf9-865b-4e2c-9bd1-a0ed0cebea8e,preventive maintenance,honda,2012


In [59]:
master_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37548 entries, 0 to 37547
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Primary Key       37548 non-null  int64  
 1   Created           37548 non-null  object 
 2   Mileage In        37548 non-null  int64  
 3   Work Order        37548 non-null  int64  
 4   Model             37548 non-null  object 
 5   Line Type         37548 non-null  object 
 6   Service Category  37548 non-null  object 
 7   Service Package   37525 non-null  object 
 8   Description       37548 non-null  object 
 9   Definition        15582 non-null  object 
 10  Price             37548 non-null  float64
 11  Quantity          37548 non-null  float64
 12  Total             37548 non-null  float64
 13  Discount          37548 non-null  float64
 14  Net Total         37548 non-null  float64
 15  ContactID         37548 non-null  object 
 16  ServiceItemID     37548 non-null  object

In [60]:
master_df['labels'].value_counts()

preventive maintenance                          15899
change oil, lube and flushing                    4957
complete vehicle inspection                      2799
steering and wheel alignment                     1962
Brakes                                           1590
suspension                                       1252
Air Conditioner General Cleaning Service         1131
clutch general services                           978
Engine                                            928
tire general services and repair                  851
Ignition and fuel general services                808
electrical general services and replacement       625
Valve Cover Gasket                                545
Cooling Repair, charging and replacement          515
Transmission general services                     430
Vacuum/Charging Service                           369
Radiator general services                         349
Wheel Bearing                                     230
Drive train general services

In [61]:
# car labels; getting only the entries with greater than 250
label_list = master_df['labels'].value_counts().to_frame()
label_list = label_list[label_list['labels'] <= 100]
label_list = list(label_list.index)
label_list

['Door Servicing',
 'Counter / Cash Sale',
 'Body Repair and Paint Works',
 'Accessories General Service',
 'Other Sublet Services (Machine Shop)',
 'Car Horn Repair & Services',
 'Car Hood Services',
 'Towing',
 'Exhaust General Service',
 'electrical general services',
 'Car Seat Services']

In [62]:
master_df = master_df[~master_df['labels'].isin(label_list)]

In [63]:
# car make; getting only the entries with greater than 100 
car_list = master_df['Make'].value_counts().to_frame()
car_list = car_list[car_list['Make'] <= 100]
car_list = list(car_list.index)
car_list

['audi',
 'mercedes benz',
 'lexus',
 'volvo',
 'volkswagen',
 'dodge',
 'lincoln',
 'foton',
 'chrysler',
 'peugeot',
 'jeep',
 'mg']

In [64]:
master_df = master_df[~master_df['Make'].isin(car_list)]

In [65]:
master_df = master_df[master_df['Service Category'] != 'Accessories (Gauges, Power Accessories, Vision & Air Bag)']

In [66]:
master_df = master_df[master_df['labels'] != 'preventive maintenance']
master_df = master_df[master_df['labels'] != 'Engine Detailing']

# 
master_df = master_df[master_df['labels'] != 'Drive train general services and replacement']
master_df = master_df[master_df['labels'] != 'filter and sensor (air and fuel)']

In [67]:
labl = master_df['labels'].value_counts().to_frame()
labl

,labels
"change oil, lube and flushing",4924
complete vehicle inspection,2766
steering and wheel alignment,1930
Brakes,1575
suspension,1238
Air Conditioner General Cleaning Service,1103
clutch general services,978
Engine,918
tire general services and repair,840
Ignition and fuel general services,762


In [68]:
master_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20218 entries, 13 to 37547
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Primary Key       20218 non-null  int64  
 1   Created           20218 non-null  object 
 2   Mileage In        20218 non-null  int64  
 3   Work Order        20218 non-null  int64  
 4   Model             20218 non-null  object 
 5   Line Type         20218 non-null  object 
 6   Service Category  20218 non-null  object 
 7   Service Package   20196 non-null  object 
 8   Description       20218 non-null  object 
 9   Definition        7843 non-null   object 
 10  Price             20218 non-null  float64
 11  Quantity          20218 non-null  float64
 12  Total             20218 non-null  float64
 13  Discount          20218 non-null  float64
 14  Net Total         20218 non-null  float64
 15  ContactID         20218 non-null  object 
 16  ServiceItemID     20218 non-null  objec

In [69]:
X = master_df[['Year', 'Make', 'Model','Mileage In']]
Y = master_df['labels']

In [70]:
# category encoding
# !pip install category_encoders


encoder = ce.BinaryEncoder(cols=['Year', 'Make', 'Model'], return_df = True)

In [71]:
encoder.fit(X)

BinaryEncoder(cols=['Year', 'Make', 'Model'])

# Splitting of Dataset

In [72]:
x_train, x_test, y_train, y_test = train_test_split(X, Y, random_state = 4244, test_size = 0.20, stratify = Y)

In [73]:
X_train_encoded = encoder.transform(x_train)
X_test_encoded = encoder.transform(x_test)

In [74]:
print(X_train_encoded.shape, X_test_encoded.shape)

(16174, 22) (4044, 22)


In [75]:
# Using gridsearch for RandomForestClassifier

In [76]:
# model_ = RandomForestClassifier(random_state = 4244, n_jobs = 4)

In [92]:
param1 = {
    'n_estimators': [50, 100, 150, 200],
    'criterion': ['gini','entropy'],
    'max_depth': [5, 10, 15, 20, 25],
    'max_features': ['auto', 'sqrt'],
    'bootstrap': [True, False],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf' : [1, 2, 4]    
}

"param1 = {\n    'n_estimators': [50, 100, 150, 200],\n    'criterion': ['gini','entropy'],\n    'max_depth': [5, 10, 15, 20, 25],\n    'max_features': ['auto', 'sqrt'],\n    'bootstrap': [True, False],\n    'min_samples_split': [2, 5, 10],\n    'min_samples_leaf' : [1, 2, 4]    \n}"

In [78]:
# grid_model = GridSearchCV(model_, param1, cv = 5)

In [79]:
# grid_model.fit(X_train_encoded, y_train)

In [80]:
# print(grid_model.best_params_)

In [81]:
# y_pred = grid_model.predict(X_test_encoded)

In [82]:
# print("Precision score: {:.3f}".format(precision_score(y_test,y_pred , average='macro')))
# print("Accuracy Score:{:.3f}".format(accuracy_score(y_test, y_pred)))
# print("Classification Report: \n", classification_report(y_test, y_pred))

In [83]:
# rf_model = RandomForestClassifier(n_estimators = 150, bootstrap = False, criterion = 'entropy', max_depth = 25, max_features = 'auto', min_samples_leaf = 1, min_samples_split = 2, random_state = 4244)

In [84]:
# rf_model.fit(X_train_encoded, y_train)

In [85]:
# y_pred = grid_model.predict(X_test_encoded)

In [86]:
# print("Precision score: {:.3f}".format(precision_score(y_test,y_pred , average='macro')))
# print("Accuracy Score:{:.3f}".format(accuracy_score(y_test, y_pred)))
# print("Classification Report: \n", classification_report(y_test, y_pred))

In [88]:
# Using AdaboostClassifier (default)

random_forest_model = RandomForestClassifier(n_estimators = 150, bootstrap = False, criterion = 'entropy', max_depth = 25, max_features = 'auto', min_samples_leaf = 1, min_samples_split = 2, random_state = 4244, n_jobs = 4)
abc_model = AdaBoostClassifier(base_estimator=random_forest_model, random_state = 4244)

In [89]:
abc_model.fit(X_train_encoded, y_train)

AdaBoostClassifier(base_estimator=RandomForestClassifier(bootstrap=False,
                                                         criterion='entropy',
                                                         max_depth=25,
                                                         n_estimators=150,
                                                         n_jobs=4,
                                                         random_state=4244),
                   random_state=4244)

In [90]:
y_pred = abc_model.predict(X_test_encoded)

In [91]:
print("Precision score: {:.3f}".format(precision_score(y_test,y_pred , average='macro')))
print("Accuracy Score:{:.3f}".format(accuracy_score(y_test, y_pred)))
print("Classification Report: \n", classification_report(y_test, y_pred))

Precision score: 0.652
Accuracy Score:0.709
Classification Report: 
                                              precision    recall  f1-score   support

   Air Conditioner General Cleaning Service       0.77      0.81      0.79       221
    Alternator Diagnosis / Component Repair       0.70      0.48      0.57        29
                                     Brakes       0.68      0.65      0.66       315
   Cooling Repair, charging and replacement       0.62      0.66      0.64       103
                                     Engine       0.69      0.67      0.68       184
         Ignition and fuel general services       0.58      0.44      0.50       152
                  Radiator general services       0.58      0.56      0.57        70
              Transmission general services       0.60      0.52      0.56        85
                    Vacuum/Charging Service       0.66      0.58      0.62        72
                         Valve Cover Gasket       0.55      0.53      0.54      